# PIV computation

## Launching the topology of computation for PIV

In [ ]:
import os
os.environ["OMP_NUM_THREADS"] = "1"

In [ ]:
from fluidimage import path_image_samples

In [ ]:
path_src = path_image_samples / "Karman/Images"
postfix = "doc_piv_ipynb"

In [ ]:
from fluidimage.topologies.piv import TopologyPIV

We use a class function to create an object containing the parameters.

In [ ]:
params = TopologyPIV.create_default_params()

The representation of this object is useful. In Ipython, just do:

In [ ]:
params

We here see a representation of the default parameters. Some elements have a `_doc` attribute. For example:

In [ ]:
params.multipass._print_doc()

We can of course modify these parameters. An error will be raised if we accidentally try to modify a non existing parameter. We at least need to give information about where are the input images:

In [ ]:
params.series.path = str(path_src)
params.series.ind_start = 1

params.piv0.shape_crop_im0 = 64
params.multipass.number = 2
params.multipass.use_tps = False

params.saving.how = 'recompute'
params.saving.postfix = postfix

In [ ]:
topology = TopologyPIV(params)

We then launch the computation by running the function `topology.compute`. We use a sequential executor to get a simpler logging.

In [ ]:
topology.compute(sequential=True)

In [ ]:
path_src

In [ ]:
topology.path_dir_result

## Analyzing the computation

In [ ]:
from fluidimage.topologies.log import LogTopology
log = LogTopology(topology.path_dir_result)

In [ ]:
log.durations

In [ ]:
log.plot_durations()

In [ ]:
log.plot_memory()

In [ ]:
log.plot_nb_workers()

## Loading the output files

In [ ]:
cd $topology.path_dir_result

In [ ]:
from fluidimage import create_object_from_file

In [ ]:
o = create_object_from_file('piv_01-02.h5')

In [ ]:
print([s for s in dir(o) if not s.startswith('_')])

In [ ]:
print([s for s in dir(o.piv1) if not s.startswith('_')])

In [ ]:
o.display()

In [ ]:
o.piv0.display(show_interp=False, scale=0.1, show_error=False, pourcent_histo=99, hist=False, show_correl=True)

The output PIV files are just hdf5 files. If you just want to load the final velocity field, do it manually with h5py.

In [ ]:
import h5py
with h5py.File('piv_01-02.h5') as f:
    deltaxs_final = f['piv1/deltaxs_final'][:]

In [ ]:
from shutil import rmtree
rmtree(topology.path_dir_result, ignore_errors=True)

In [ ]:
cd $path_src